In [ ]:
from transformers import pipeline 
from PIL import Image,ImageDraw, ImageFont
import requests
from IPython.display import display
import cv2 
import numpy as np
import sys
import os
import torch
import shutil

# Load Object-Detection Model ( model = pipeline(model="hustvl/yolos-tiny", device="cuda:0" for gpu usage)
model = pipeline(model="hustvl/yolos-tiny", device="cuda:0")

In [ ]:
# Usage: ( Sort Videos based on whether the video contains a human or not )
# Import necessary libraries and load the model
# Load definitions
# Select folder to analyse ( folder_path = )
# Outputs human and nothuman folders to load individual frames of each video
# Processes frames to determine whether video contains human or not
# Once a human is detected in a frame, move the video to human_vid 
# Else if theres no human, move it to nothuman_vid

In [ ]:
def object_detector(file,file_path):
    workspace = '/home/jovyan'
    # For deleting folders
    def delete_folder(folder_path):
        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            shutil.rmtree(folder_path)
            print(f"Folder '{folder_path}' has been deleted successfully.")
        else:
            print(f"Error: Folder '{folder_path}' does not exist.")

    # Deletes folders
    delete_folder('output_frames')


    # Video capture
    cap = cv2.VideoCapture(f"{file_path}")

    # Creates Output folder for saving frames
    output_folder = "output_frames"
    os.makedirs(output_folder, exist_ok=True)
    # Labels
    frame_count = 0

    while True:
        # Reads each frame 
        success, img = cap.read()
        if not success:
            break  # Break out of the loop if there are no more frames to read

        # Save the current frame as an image to output folder
        output_path = os.path.join(output_folder, f"frame_{frame_count:06d}.jpg")
        cv2.imwrite(output_path, img)

        frame_count += 1

    # Release video capture
    cap.release()
    # Number of frames in the video
    print(f"Total frames saved: {frame_count}")


    # Loads the images
    def load_images_from_folder(folder_path):
        images = []
        file_name = []
        # Goes through the folder
        for filename in os.listdir(folder_path):
            # If the file is an image
            if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add any other image file extensions you need
                image_path = os.path.join(folder_path, filename)
                try:
                    image = Image.open(image_path)
                    # Adds the image to the images list 
                    images.append(image)
                    # Records the file-name loaded
                    file_name.append(filename)
                except Exception as e:
                    print(f"Failed to load image {filename}: {e}")
        return images, file_name

    # Split list into chunks to ensure processor doesnt crash
    def chunk_list(lst, chunk_size):
        for i in range(0, len(lst), chunk_size):
            yield lst[i:i + chunk_size]

    # Ensure output directories exist
    if not os.path.exists('human_vid'):
        os.makedirs('human_vid')
    if not os.path.exists('nothuman_vid'):
        os.makedirs('nothuman_vid')
    # Folder path of the images
    folder_path_images = output_folder
    # Loads the images to a list 
    loaded_images,file_name = load_images_from_folder(folder_path_images)
    
    # Process images in batches
    chunk_size = 50
    success = False 
    # Loops for every x images
    for i, image_chunk in enumerate(chunk_list(loaded_images, chunk_size)):
        print(f"Processing batch {i + 1}")
        # Sends x images to the  model to process
        detections = model(image_chunk)
        # Loops x number of times for each batch 
        for j in range(len(image_chunk)):       
            for detection in detections[j]:
                # If a person is detected in the image, move video to human_vid folder
                if detection['score'] > 0.9 and detection['label'] == "person":
                    success = True
                    # Output path to human_vid
                    output_video_path =f"{workspace}/human_vid/{file}"
                    shutil.move(file_path,output_video_path)
                    print("Human Detected.")
                    break
            if success == True:
                break
        if success == True:
            break 
    # If no human detected, move it to nothuman_vid
    if not success:
        # Output path
        output_video_path =f"{workspace}/nothuman_vid/{file}"
        shutil.move(file_path,output_video_path)
        print("No Human Detected.")
        
    print("Processing complete.")
    print(f"Video created successfully, File:{output_video_path}")

In [ ]:
def process_videos(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    
    # Loop through each file
    for file in files:
        # Check if the file is a video file based on its extension
        if file.endswith(('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv')):
            # Full file path
            file_path = os.path.join(folder_path, file)
            
            # Print the file name (or perform any other processing)
            print(f"Processing video file: {file}")
            
            # TODO: Add your video processing code here
            object_detector(file, file_path)
        print("Sorting Completed.")
# Specify the folder containing the videos
folder_path = 'Videos'

# Call the function to process videos
process_videos(folder_path)